In [ ]:
# install requirements from requirements_fingerprinting.txt
%pip install -r requirements_fingerprinting.txt

In [93]:
import data_sanitize
import importlib
importlib.reload(data_sanitize)
import pandas as pd

training_data = data_sanitize.get_training_data()

In [95]:
# convert the training data which is a numpy array
            # self.label,
            # self.num_outgoing,
            # self.num_incoming,
            # self.total_packets,
            # self.outgoing_ratio,
            # self.incoming_ratio,
            # self.outgoing_bytes,
            # self.incoming_bytes,
            # self.avg_outgoing_freq,
            # self.avg_incoming_freq,
            # self.std_outgoing_freq,
            # self.std_incoming_freq,
            # self.avg_outgoing_bytes,
            # self.avg_incoming_bytes,
            # self.std_outgoing_bytes,
            # self.std_incoming_bytes,
            # self.min_outgoing_bytes,
            # self.min_incoming_bytes,
            # self.max_outgoing_bytes,
            # self.max_incoming_bytes,
            # self.min_outgoing_freq,
            # self.min_incoming_freq,
            # self.max_outgoing_freq,
            # self.max_incoming_freq,
            # self.avg_outgoing_order,
            # self.std_outgoing_order,
df = pd.DataFrame(training_data, columns=[
    'grid',
    'num_outgoing',
    'num_incoming',
    'total_packets',
    'outgoing_ratio',
    'incoming_ratio',
    'outgoing_bytes',
    'incoming_bytes',
    'avg_outgoing_freq',
    'avg_incoming_freq',
    'std_outgoing_freq',
    'std_incoming_freq',
    'avg_outgoing_bytes',
    'avg_incoming_bytes',
    'std_outgoing_bytes',
    'std_incoming_bytes',
    'min_outgoing_bytes',
    'min_incoming_bytes',
    'max_outgoing_bytes',
    'max_incoming_bytes',
    'min_outgoing_freq',
    'min_incoming_freq',
    'max_outgoing_freq',
    'max_incoming_freq',
    #'avg_outgoing_order',
    #'std_outgoing_order',
])
df.head()

,grid,num_outgoing,num_incoming,total_packets,outgoing_ratio,incoming_ratio,outgoing_bytes,incoming_bytes,avg_outgoing_freq,avg_incoming_freq,...,std_outgoing_bytes,std_incoming_bytes,min_outgoing_bytes,min_incoming_bytes,max_outgoing_bytes,max_incoming_bytes,min_outgoing_freq,min_incoming_freq,max_outgoing_freq,max_incoming_freq
0,49.0,168.0,135.0,305.0,0.550820,0.442623,322478.0,31992.0,0.109459,0.136780,...,1632.681136,371.849979,52.0,52.0,7292.0,2948.0,0.000009,0.000008,1.864979,1.914120
1,49.0,179.0,150.0,331.0,0.540785,0.453172,371718.0,34402.0,0.079143,0.094551,...,1802.563964,413.940833,52.0,52.0,7292.0,4122.0,0.000021,0.000064,0.911165,0.960341
2,49.0,150.0,128.0,292.0,0.513699,0.438356,311358.0,35380.0,0.109650,0.129042,...,1782.315782,383.764931,52.0,52.0,7292.0,2948.0,0.000010,0.000018,1.226477,1.226678
3,49.0,163.0,130.0,295.0,0.552542,0.440678,312790.0,32290.0,0.082891,0.104447,...,1608.133336,372.432015,52.0,52.0,7292.0,2948.0,0.000024,0.000023,1.019120,1.037091
4,49.0,167.0,127.0,297.0,0.562290,0.427609,346352.0,32134.0,0.116166,0.153365,...,1904.074364,375.565662,52.0,52.0,10188.0,2948.0,0.000011,0.000026,2.197595,1.951306


In [75]:
# what are the unique grids in the dataset?
label_count = len(df['grid'].unique())
label_count

100

In [97]:
# we want to train a model to predict the grid number based on the other features
# we will use a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

# we'll use grid search to find the best hyperparameters for the model
from sklearn.model_selection import GridSearchCV

# we'll use a pipeline to chain the preprocessing and the model
from sklearn.pipeline import Pipeline

# we'll use a standard scaler to scale the features
from sklearn.preprocessing import StandardScaler

# start by splitting the data into training and testing sets
X = df.drop('grid', axis=1)
y = df['grid']

# stratify the data to ensure that the training and testing sets have the same distribution of grids
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print("Before feature selection: ", X_train.shape)
selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
print("After feature selection: ", X_train.shape)

# fit a random forest classifier to the training data
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# evaluate the model on the testing data
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

Before feature selection:  (4000, 23)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


After feature selection:  (4000, 10)


0.751

In [80]:
len(df.columns)

24

In [ ]:
# we want to train a model to predict the grid number based on the other features
# we will use a random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

# we'll use grid search to find the best hyperparameters for the model
from sklearn.model_selection import GridSearchCV

# we'll use a pipeline to chain the preprocessing and the model
from sklearn.pipeline import Pipeline

# we'll use a standard scaler to scale the features
from sklearn.preprocessing import StandardScaler

# start by splitting the data into training and testing sets
X = df.drop('grid', axis=1)
y = df['grid']

# stratify the data to ensure that the training and testing sets have the same distribution of grids
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print("Before feature selection: ", X_train.shape)
selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
print("After feature selection: ", X_train.shape)

# create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
])

# create the parameter grid
param_grid = {
    'clf__n_estimators': [100, 200, 300, 400, 500],
    'clf__max_depth': [None, 5, 10, 15, 20, 25, 30],
    'clf__min_samples_split': [2, 5, 10, 15, 20],
    'clf__min_samples_leaf': [1, 2, 5, 10, 15, 20],
    'clf__max_features': ['sqrt', 'log2', None]
}

# create the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1)

# fit the grid search
grid_search.fit(X_train, y_train)

In [ ]:
# print the best parameters, with feature selection: selector = SelectFromModel(RandomForestClassifier().fit(X_train, y_train), prefit=True)
print("Best params: ", grid_search.best_params_)

# print the best score
print("Best score: ", grid_search.best_score_)

# print the accuracy on the test set
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
# print the best parameters
print(grid_search.best_params_)

# print the best score
print(grid_search.best_score_)

print(accuracy_score(y_test, y_pred))

In [ ]:
# we now try MLP classifier
from sklearn.neural_network import MLPClassifier

# create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', MLPClassifier())
])

# create the parameter grid
param_grid = {
    'clf__hidden_layer_sizes': [(100,), (200,), (300,), (400,), (500,)],
    'clf__activation': ['identity', 'logistic', 'tanh', 'relu'],
    'clf__solver': ['lbfgs', 'sgd', 'adam'],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'clf__learning_rate': ['constant', 'invscaling', 'adaptive']
}

# create the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)

# fit the grid search
grid_search.fit(X_train, y_train)

# print the best parameters
grid_search.best_params_

# print the best score
grid_search.best_score_

# print the accuracy on the test set
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
# We know try a KNN classifier
from sklearn.neighbors import KNeighborsClassifier

# create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

# create the parameter grid
param_grid = {
    'clf__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'clf__weights': ['uniform', 'distance'],
    'clf__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'clf__leaf_size': [10, 20, 30, 40, 50],
    'clf__p': [1, 2]
}

# create the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)

# fit the grid search
grid_search.fit(X_train, y_train)

# print the best parameters
grid_search.best_params_

# print the best score
grid_search.best_score_

# print the accuracy on the test set
y_pred = grid_search.predict(X_test)